In [ ]:
import geopandas as gpd
import pygeos as pygeos

from geopy import distance
from sklearn.preprocessing import LabelEncoder

import pandas as pd
from simpledbf import Dbf5

In [ ]:
def calc_dist(df):
    x, y = df['centroid_x'], df['centroid_y']
    
    x_nn, y_nn = df['nn_centroid_x'], df['nn_centroid_y']
    
    return distance.distance((y,x), (y_nn, x_nn)).meters

In [ ]:
roads_shp = "portugal-latest-free.shp 2/gis_osm_roads_free_1.shp"

In [ ]:
roads = gpd.read_file(roads_shp)

In [ ]:
freedom_roads = roads[(roads.name.str.contains("25 de Abril")) | (roads.name.str.contains("Liberdade")) | (roads.name.str.contains("Salgueiro Maia")) | (roads.name.str.contains("Capitães de Abril"))]

In [ ]:

df2 = freedom_roads[['osm_id', 'name', 'geometry']]
df2 = df2.rename(columns={'name': 'rua'})

In [ ]:
df2.head(3)

In [ ]:
concelhos = gpd.read_file('concelhos-shapefile/concelhos.shp', SHAPE_RESTORE_SHX='YES')

concelhos = concelhos.rename(columns={'NAME_2': 'concelho'})

df2['centroid_json'] = df2.geometry.apply(lambda x: x.centroid)
df2 = df2.sjoin(concelhos[['concelho', 'geometry']], how="left")

In [ ]:
freguesias = gpd.read_file('cont-aad-caop2017/Cont_AAD_CAOP2017.shp', SHAPE_RESTORE_SHX='YES')

In [ ]:
df2 = df2.drop(columns=['index_right'])

In [ ]:
freguesias.to_crs(epsg=4326, inplace=True)

In [ ]:
df2 = df2.sjoin(freguesias[['Freguesia', 'geometry']], how="left")

In [ ]:
# Spatial join leading to some duplicates
df2 = df2.drop_duplicates('osm_id')

In [ ]:
df2['centroid_x'] = df2.geometry.apply(lambda x: x.centroid.x)
df2['centroid_y'] = df2.geometry.apply(lambda x: x.centroid.y)

In [ ]:
df2.Freguesia = df2.Freguesia.fillna('')

In [ ]:
df2.head(3)

In [ ]:
# m = folium.Map(location=[39.9, -8.22])

#for i, row in df2.iterrows():
#    folium.Marker([row['centroid_y'], row['centroid_x']]).add_to(m)


In [ ]:
df2['rua_freguesia_concelho'] = df2['rua'] + "_" +  df2['Freguesia'] + "_" + df2['concelho']
df2

In [ ]:
le = LabelEncoder()
df2['unique_index'] = le.fit_transform(df2.rua_freguesia_concelho)
df2 = df2.drop(columns=['centroid_json'])
df2

In [ ]:
df2.to_file('ruas_abril.geojson', driver='GeoJSON')